In [1]:
import os
os.chdir('/home/roberto/Documents/Titulación/Tesina')
from scripts import *

### Países disponibles:

##### -España
##### -Alemania

### Años disponibles:

##### 2008 a 2017
### Métodos disponibles

correr_simulacion() para ejecutar simulación de nuevo

leer() leer como DataFrame la salida de la simulación

area_elegible(country, includer) para mapa cuadriculado del área a utilizar para paneles (Includer disponibles = dump y water)

tres_random(pais) para gráfica de tres días aleatorios de generación de energía

anual_country(pais) Generación anual por país. Regresa DF

suma_mes(country,mes) Regresa la producción total de un mes para cada año en un país (Regresa diccionario)

graficas_bienestar(country) Diagrama de caja y brazos para un país

produccion_por_mes(country) Producción por mes. Regresa la media y su estimación con intervalos de confianza al 95% (DataFrame)

ventas_spain() Regresa DataFrame con las ventas por cada mes de España

ventas_germany() Regresa DataFrame con las ventas por cada mes de Alemania

graficas_cuatrimestre(country) Gráficas por cuatrimestre. Perc.0.75, median y perc. 0.25

In [2]:
ventas_germany()

,Precios (Euros/MWh),Produccion [GWh],Ventas (Euros)
January,€49.39,56.113099,"€2,771,425.94"
February,€42.82,84.109818,"€3,601,582.43"
March,€30.63,140.619124,"€4,307,163.78"
April,€39.96,175.475384,"€7,011,996.35"
May,€37.84,181.67163,"€6,874,454.48"
June,€32.52,176.150603,"€5,728,417.60"
July,€39.69,180.758553,"€7,174,306.96"
August,€36.85,169.920202,"€6,261,559.43"
September,€35.75,133.149721,"€4,760,102.52"
October,€36.94,97.604131,"€3,605,496.62"


In [3]:
ventas_spain()

,Precios (Euros/MWh),Produccion [GWh],Ventas (Euros)
January,€48.21,118.824411,"€5,728,524.85"
February,€42.61,134.036667,"€5,711,302.37"
March,€35.03,180.477737,"€6,322,135.14"
April,€27.38,192.118765,"€5,260,211.78"
May,€28.98,208.482249,"€6,041,815.57"
June,€37.54,207.848842,"€7,802,645.52"
July,€41.10,220.886161,"€9,078,421.20"
August,€43.12,212.815911,"€9,176,622.10"
September,€49.30,181.239021,"€8,935,083.73"
October,€44.63,157.986702,"€7,050,946.52"
